# Second submission: Random Forest
 * min observations in each category: 2
 * best parameters:  



In [3]:
%matplotlib inline
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# set seeds
np.random.seed(1)
random.seed(1)

# Clean data

## Create and drop features, manage categorical

In [4]:
# Function to prepare data

from sklearn.base import TransformerMixin

def create_new_features(X):
    # transform utc offset:
    X['Sin_UTC'] = np.sin((11 * 3600 + X['UTC Offset']) / (24 * 3600) * 2 * np.pi)
    X['Cos_UTC'] = np.cos((11 * 3600 + X['UTC Offset']) / (24 * 3600) * 2 * np.pi)
    
    # time since creation in days
    duration = (pd.to_datetime('today') - 
                pd.to_datetime(X['Profile Creation Timestamp']).dt.tz_localize(None))
    X['Duration'] = duration.apply(lambda x: x.days)
    
    # convert personal url into True/False (NaNs or unique)
    X['Has Personal URL'] = X['Personal URL'].notna()
    
    # log(x + 1) of numerical features
    for feature in ['Num of Followers', 'Num of People Following',
                    'Num of Status Updates', 'Num of Direct Messages',
                    'Avg Daily Profile Visit Duration in seconds',
                    'Avg Daily Profile Clicks']:
        X[f'log {feature}'] = np.log1p(X[feature])
    return X
    
def clean_features(X):
    # merge categories names with and without cap letter
    X['Location Public Visibility'] = X['Location Public Visibility'].str.lower()
    X.loc[X['Profile Category'] == " ", 'Profile Category'] = 'unknown'
    return X

def drop_features(X, l_features):
    return X.drop(columns=l_features)
    
class RemoveCategories(TransformerMixin):
    def __init__(self, min_obs=10, l_cols=[]):
        self.min_obs = min_obs
        self.l_cats = {}
        self.l_cols = l_cols

    def fit(self, X):
        # assumes all columns of df_cat are strings
        for col in self.l_cols:
            val_counts = X[col].fillna('missing').value_counts()
            self.l_cats[col] = val_counts[val_counts >= self.min_obs].index.tolist()
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        for col in self.l_cols:
            X.loc[~ X[col].isin(self.l_cats[col]), col] = 'other'        
        return X

In [5]:
# data cleaning and feature engineering

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

l_cat_small = ['Profile Text Color', 'Profile Page Color',
               'Profile Theme Color', 'User Language', 'Location']
l_drop = ['Id', 'User Name', 'Profile Image', 'Personal URL',
          'Profile Creation Timestamp', 'Location', 'UTC Offset']
l_drop_logs = ['Num of Followers', 'Num of People Following',
               'Num of Status Updates', 'Num of Direct Messages',
               'Avg Daily Profile Visit Duration in seconds', 
               'Avg Daily Profile Clicks']

features_to_drop =dict(l_features = l_drop + l_drop_logs)

pipe = Pipeline([('create new features',
                  FunctionTransformer(create_new_features)),
                 ('clean data', FunctionTransformer(clean_features)),
                 ('remove categories', RemoveCategories(min_obs=2, 
                                                        l_cols=l_cat_small)),
                 ('drop variables', FunctionTransformer(drop_features,
                                    kw_args=features_to_drop))
                ])

## Imputation and encoding/scaling

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
    ])

# categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [7]:
# train data
df = pd.read_csv('data/train.csv')
y = np.log1p(df['Num of Profile Likes'].values)
X = df.drop(columns='Num of Profile Likes')

X = pipe.fit_transform(X)

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer,
                   X.select_dtypes(exclude=['object', 'bool']).columns),
                  ('cat', categorical_transformer,
                   X.select_dtypes(include=['object', 'bool']).columns)])

X = preprocessor.fit_transform(X)

# Random Forest

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(criterion='mse', n_jobs=-1)

parameters = {'n_estimators': [100, 200, 300],
              'max_depth': range(2, 20),
              'min_samples_split': range(2, 10),
              'min_samples_leaf': range(2, 10)}

reg_rf = RandomizedSearchCV(rf, parameters, scoring='neg_mean_squared_error',
                      n_jobs=-1, random_state=0, n_iter=20)
reg_rf.fit(X, y)

print(f'RMSLE: {np.sqrt(-reg_rf.best_score_):.3f}',
      f'\nbest parameters: { reg_rf.best_params_}')

RMSLE: 1.739 
best parameters: {'n_estimators': 300, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_depth': 13}


# Submission

In [1]:
def create_submission(estimator, number):
    X_test = pd.read_csv('data/test.csv')
    Id = X_test['Id']
    X_test = pipe.transform(X_test)
    X_test = preprocessor.transform(X_test)
    y_pred = (estimator.predict(X_test))
    y_pred_test = pd.Series(np.expm1(y_pred))
    submission = pd.DataFrame({'Id': Id, 'Predicted': y_pred_test})
    path = f'submissions/submission{number}.csv'
    while os.path.exists(path):
        number += 1
        path = f'submissions/submission{number}.csv'
    submission.to_csv(path, index=False)
    print(f'submission #{number} created')

In [2]:
create_submission(reg_rf, 1)

NameError: name 'reg_rf' is not defined

In [ ]:
pd.read_csv('submissions/submission2.csv').head()